In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import ttest_rel, wilcoxon

In [2]:
subs = ['sub-01','sub-02', 'sub-03','sub-04', 'sub-05', 'sub-06']
ft_models = ['conv4', 'no']
atlas = ['wholebrain', 'STG']
baseline = 'no'

dir_path = '/home/maellef/Results/one_run_eval_friends_2024'

dir_name = 'data_from_'

In [3]:
df_all = pd.DataFrame(columns = ['model_sub', 'data_sub', 'scale', 'conv', 'r²'])
for sub in subs:
    sub_path = os.path.join(dir_path, dir_name+sub)
    data_sub = sub
    print(data_sub)
    for model in os.listdir(sub_path):
        model_path = os.path.join(sub_path, model)
        r2_array = np.load(model_path)
        r2max_array = np.max(r2_array, axis=1)
        r2mean_array = np.mean(r2_array, axis=1)
        r2STG_array = r2_array[:,153]
        
        if 'group_model' not in model:
            model_sub = model[:len('sub-0')+1]
        else:
            model_sub = 'group model'
            #i = model.index('group_model_')
            #l = len('group_model_')+5
            #model_sub = model[i:i+l]
        
        scale = 'whole brain' if 'MIST_ROI' in model else 'stg'
        finetune_cond = model.find('_f_') 
        conv = model[finetune_cond+3:finetune_cond+3+len('conv')+1] if finetune_cond>-1 else 'no'
        
        model_data = {
            'model_sub':np.full(len(r2max_array), model_sub),
            'data_sub':np.full(len(r2max_array),data_sub), 
            'scale':np.full(len(r2max_array),scale), 
            'conv':np.full(len(r2max_array),conv), 
            'r²': r2STG_array
                     }
        model_df = pd.DataFrame(model_data)
        df_all = pd.concat((df_all, model_df), axis='rows')
        #print('data: ',sub,'model: ',model_sub, 'conv: ', conv, 'scale: ', scale)

df = df_all.loc[df_all['conv'].isin(['conv4', 'no'])]

sub-01
sub-02
sub-03
sub-04
sub-05
sub-06


/tmp/ipykernel_41156/475973922.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat((df_all, model_df), axis='rows')


In [4]:
full_subs = subs.copy()
full_subs.append('group model')
print(full_subs)

for scale in ['whole brain', 'stg']:
    for conv in ['conv4']:
        for data_sub in subs:
            sub_df = df.loc[(df['scale']==scale)&(df['conv']==conv)&(df['data_sub']==data_sub)&(df['model_sub']==data_sub)]
            sub_r2 = sub_df['r²'].values
            for model_sub in full_subs :
                other_df = df.loc[(df['scale']==scale)&(df['conv']==conv)&(df['data_sub']==data_sub)&(df['model_sub']==model_sub)]
                diff_r2 = [None]*len(sub_r2) if model_sub == data_sub else other_df['r²'].values - sub_r2
                
                df.loc[(df['scale']==scale)&(df['conv']==conv)&(df['data_sub']==data_sub)&(df['model_sub']==model_sub), 'diff r²'] = diff_r2
                
print(df.loc[df['model_sub'] == 'group model'])

['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'group model']
      model_sub data_sub        scale   conv        r²   diff r²
0   group model   sub-01          stg  conv4 -0.013430 -0.103492
1   group model   sub-01          stg  conv4  0.446628 -0.016307
2   group model   sub-01          stg  conv4 -0.009327 -0.046218
3   group model   sub-01          stg  conv4 -0.037004 -0.067266
4   group model   sub-01          stg  conv4  0.052341  0.003797
..          ...      ...          ...    ...       ...       ...
43  group model   sub-06  whole brain  conv4  0.186029  0.010949
44  group model   sub-06  whole brain  conv4  0.265740 -0.008561
45  group model   sub-06  whole brain  conv4  0.253306   0.01795
46  group model   sub-06  whole brain  conv4  0.210884  0.020366
47  group model   sub-06  whole brain  conv4  0.167019  0.002172

[568 rows x 6 columns]


In [5]:
#significative difference between prediction of subjects signals for a scale and a conv model
print(df.columns)
for scale in ['whole brain', 'stg']:
    for conv in ['conv4']:
        for data_sub in subs : 
            df_sub = df.loc[(df['data_sub'] == data_sub) & (df['scale'] == scale) & (df['conv'] == conv)]
            best_model_sub_df = df_sub['r²'].loc[df_sub['model_sub'] == data_sub]
            print(scale, data_sub, conv)
            for model_sub in full_subs:
                if model_sub == data_sub : 
                    pass
                else : 
                    #print(scale, data_sub, model_sub, conv)
                    model_sub_df = df_sub['diff r²'].loc[df_sub['model_sub'] == model_sub]
                    x = model_sub_df.values.astype(np.float64)
                    median = np.median(x)
                    stat, pvalue = wilcoxon(x)
                    sign_test = 'success' if pvalue < 0.05 else 'fail'
                    print('pvalue between {} and {} on data from {} is {} : {}, median: {}'.format(data_sub, model_sub, 
                                                                                  data_sub, pvalue, sign_test, median))

Index(['model_sub', 'data_sub', 'scale', 'conv', 'r²', 'diff r²'], dtype='object')
whole brain sub-01 conv4
pvalue between sub-01 and sub-02 on data from sub-01 is 0.009022168802360397 : success, median: -0.008788645267486572
pvalue between sub-01 and sub-03 on data from sub-01 is 0.03796332543593195 : success, median: -0.01321980357170105
pvalue between sub-01 and sub-04 on data from sub-01 is 1.7976731214730535e-12 : success, median: -0.05507165193557739
pvalue between sub-01 and sub-05 on data from sub-01 is 4.973799150320701e-13 : success, median: -0.051426976919174194
pvalue between sub-01 and sub-06 on data from sub-01 is 1.3320572236352746e-06 : success, median: -0.030521482229232788
pvalue between sub-01 and group model on data from sub-01 is 0.04424147981691107 : success, median: 0.010053098201751709
whole brain sub-02 conv4
pvalue between sub-02 and sub-01 on data from sub-02 is 0.0007592971304859475 : success, median: -0.013236284255981445
pvalue between sub-02 and sub-03 on

In [ ]:
#significative difference between prediction of baseline/conv for a sub
for scale in ['whole brain', 'stg']:
    for data_sub in subs : 
        print(scale, data_sub)
        for model_sub in subs : 
            if data_sub == model_sub : 
                pass 
            else : 
                df_conv = df.loc[(df['data_sub'] == data_sub) 
                                & (df['model_sub'] == model_sub)
                                & (df['scale'] == scale)
                                & (df['conv'] == 'conv4')]
                df_baseline = df.loc[(df['data_sub'] == data_sub) 
                                & (df['model_sub'] == model_sub)
                                & (df['scale'] == scale)
                                & (df['conv'] == 'no')]
                diff_sub = df_conv['diff r²'].values - df_baseline['diff r²'].values
                stat, pvalue_diff = wilcoxon(diff_sub)
                sign_test_diff = 'success' if pvalue_diff < 0.05 else 'fail'
                print('pvalue between conv4 and basl from model \
                    trained on {} on data from {} is {} : {}'.format(model_sub,data_sub,pvalue_diff,sign_test_diff))
                

In [ ]:
#sns.set_theme(context='poster', style='whitegrid', font_scale=2) 

sns.set(rc={'figure.figsize':(20,15), 'axes.facecolor':'w', 'axes.edgecolor': 'k', 
            'grid.color':'k', 'grid.linestyle': ':',})

dfconv = df.loc[df['conv']=='conv4'].copy()
dfconv = dfconv.reset_index(drop=True)
g = sns.catplot(data=dfconv, x='conv', y='diff r²', hue='model_sub', col='data_sub', 
                row='scale', kind='strip', dodge=True,
                order=['conv4'], row_order=['stg', 'whole brain'], col_order=subs, hue_order=full_subs, sharex=False)
plt.ylim(bottom=-0.25, top=0.15) #bottom=-0.20, top=0.05)
sns.plotting_context()
g.savefig('/home/maellef/Results/figures/heareval/individual_spec_group_STGr2.jpg') 